In [13]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import re
from tqdm.notebook import tqdm

game = 'CSGO'
scraper = sntwitter.TwitterSearchScraper(f'{game} min_replies:1 min_faves:10')

tweets = []
def pre_process(text):
    # Usuwanie linków
    text = re.sub('http://\S+|https://\S+', '', text)
    text = re.sub('http[s]?://\S+', '', text)
    text = re.sub(r"http\S+", "", text)

    # Napraw wbudowane linki
    text = re.sub('&amp', 'and', text)
    text = re.sub('&lt', '<', text)
    text = re.sub('&gt', '>', text)

    # Usuwanie znaków nowej linii
    text = re.sub('[\r\n]+', ' ', text)
    
    # Usuwanie @
    text = re.sub(r'@\w+', '', text)
    
    # Usuwanie #
    text = re.sub(r'#\w+', '', text)

    # Usuń wielokrotności spacji
    text = re.sub('\s+',' ', text)
    
    # Przełóż na lowercase
    text = text.lower()
    return text

# Przeglądamy 500 najpopularniejszych tweetów dla danego hasztagu
pbar = tqdm(total=500)
for i, tweet in enumerate(scraper.get_items()):
    data = [
        tweet.date,
        tweet.id,
        tweet.rawContent,
        tweet.user.username,
        tweet.likeCount,
        tweet.retweetCount,
        tweet.lang
    ]
    if data[4] == data[5]:
        continue
    else:
        data[2] = pre_process(data[2])
        if data[2] == ' ' or data[2] == '':
            continue
        else:
            tweets.append(data)
            pbar.update(1)
            if len(tweets) >= 500:  # Przerywamy pętlę po pobraniu 500 tweetów
                break

a=pd.DataFrame(tweets, columns=['date', 'id', 'content', 'username','like_count','retweet_count', 'language'])
a.head()



  0%|          | 0/500 [00:00<?, ?it/s]

,date,id,content,username,like_count,retweet_count,language
0,2023-03-30 09:05:03+00:00,1641365914481315840,"riegl ,woletz",while_waiting2,16,17,de
1,2023-03-30 09:01:44+00:00,1641365080582557697,勝ってメインステージ行くぞー🔥,blackwiz_csgo,106,8,ja
2,2023-03-30 08:50:33+00:00,1641362264203702272,thank you guys for 100k. i truly appreciate th...,Maniac_CSGO,89,1,en
3,2023-03-30 08:40:46+00:00,1641359800616521731,die ränge zwischen cs2 und csgo sind leider ü...,Dhalucard,41,0,de
4,2023-03-30 08:31:45+00:00,1641357531334475779,csgo ak行きたい！✌,HiNas33,20,2,ja


In [15]:
a['date'] = a['date'].dt.tz_localize(None)
a.to_excel(r'export_dataframe.xlsx', index=False)